# Damatura, Nigeria IDP Analysis
##### January 2025

In [1]:
# standard library imports
import os
import sys

# add project root to Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

# third-party imports
import rasterio
import geopandas as gpd
import numpy as np
import pandas as pd

# change to project root directory
os.chdir('../')
print("directory changes")
print(f"current working directory is:", os.getcwd())

# local imports (after changing directory)
from src.population import (
    clip_raster_with_shapefile,
    raster_to_points,
    raster_to_polygons,
    pd_calc_raster_to_polygons
)

from src.idp import kmz_to_shapefile, transform_idp_description, transform_idp_complete

directory changes
current working directory is: /Users/carolinecullinan/dev/wb/damaturu-nigeria-idp


# 0.0 Area of Interest (AOI)

In [2]:
# check coordinate system of aoi, damaturu.shp
aoi = gpd.read_file("data/raw/AOI/damaturu.shp")
print(f"coordinate system: {aoi.crs}")

coordinate system: EPSG:4326


In [42]:
# save aoi to GEOJSON for export and viz
aoi.to_file("data/processed/aoi.geojson", driver='GeoJSON')

# 1.0 Population Density (Personal Analysis)

## 1.1 Population Density Data (1 km resolution)

In [40]:
# check coordinate system of population density data for Nigeria, nga_pd__2020_1km_UNadj.tif
with rasterio.open("data/raw/nga_pd_2020_1km_UNadj.tif") as src:
    print(f"coordinate system: {src.crs}")

In [2]:
# check coordinate system of aoi, damaturu.shp
aoi = gpd.read_file("data/raw/AOI/damaturu.shp")
print(f"coordinate system: {aoi.crs}")

coordinate system: EPSG:4326


### 1.1.1 Raster to Points Conversion

In [2]:
# apply "clip_raster_with_shapefile" function to get population density (raster) 
## of Damaturu, Nigeria specifically (shapefile) and save interim "damaturu_pd" (raster) tif format
clip_raster_with_shapefile(
    "data/raw/nga_pd_2020_1km_UNadj.tif",
    "data/raw/AOI/damaturu.shp",
    "data/interim/damaturu_pd.tif"
)

In [3]:
# gain insight into raster dimensions to best choose a sample rate for raster to point conversion
## (i.e., determine how many points to sample from the raster, 
## where sample_rate = 1 means that every pixel from the raster is converted to a point)
with rasterio.open("data/interim/damaturu_pd.tif") as src:
    print(f"raster dimensions: {src.width} x {src.height}")
    print(f"total pixels: {src.width * src.height}")
    
# note: raster dimensions and total pixels are small, therefore use sample_rate = 1

raster dimensions: 17 x 13
total pixels: 221


In [8]:
with rasterio.open("data/interim/damaturu_pd.tif") as src:
    total_pixels = src.width * src.height
    target_points = 20000  # adjust based on your needs (web viz, aim for 10,000-50,000 points)
    suggested_sample_rate = int(total_pixels / target_points)
    print(f"suggested sample rate: {suggested_sample_rate}")

suggested sample rate: 0


In [4]:
# convert interim (raster) "damaturu_pd" to processed "damaturu_pd_points" points for export and viz
raster_to_points(
    "data/interim/damaturu_pd.tif",
    'data/processed/damaturu_pd_points.geojson',
    sample_rate=1
)

input raster dimensions: 17 x 13
total pixels: 221

Created 221 points
Value range: -99999.00 to 3183.82


,population_density,geometry
0,-99999.0,POINT (11.91125 11.79542)
1,-99999.0,POINT (11.91958 11.79542)
2,-99999.0,POINT (11.92792 11.79542)
3,-99999.0,POINT (11.93625 11.79542)
4,-99999.0,POINT (11.94458 11.79542)
...,...,...
216,-99999.0,POINT (12.01125 11.69542)
217,-99999.0,POINT (12.01958 11.69542)
218,-99999.0,POINT (12.02792 11.69542)
219,-99999.0,POINT (12.03625 11.69542)


### 1.1.2 Raster to Polygon Conversion

In [4]:
# convert Damaturu specific population density raster to polygon GeoJSON for export and viz
raster_to_polygons(
    "data/raw/nga_pd_2020_1km_UNadj.tif",
    "data/raw/AOI/damaturu.shp",
    "data/processed/damaturu_poly.geojson"
)


Created 95 polygons
Value range: 89.82 to 3183.82


,geometry,population_density
0,"POLYGON ((11.97375 11.79125, 11.97375 11.78292...",132.874374
1,"POLYGON ((11.98208 11.79125, 11.98208 11.78292...",118.977905
2,"POLYGON ((11.96542 11.78292, 11.96542 11.77458...",249.598358
3,"POLYGON ((11.97375 11.78292, 11.97375 11.77458...",171.849319
4,"POLYGON ((11.98208 11.78292, 11.98208 11.77458...",165.467773
...,...,...
91,"POLYGON ((11.92375 11.70792, 11.92375 11.69958...",91.995384
92,"POLYGON ((11.93208 11.70792, 11.93208 11.69958...",142.913956
93,"POLYGON ((11.94042 11.70792, 11.94042 11.69958...",224.143234
94,"POLYGON ((11.94875 11.70792, 11.94875 11.69958...",401.490784


## 1.2 Population Data (100 m resolution)

In [2]:
# calculate population density for Damaturu, Nigeria at 100 m resolution
pd_calc_raster_to_polygons(
    "data/raw/nga_ppp_2020_UNadj.tif",  # 100m population raster
    "data/raw/AOI/damaturu.shp",     #  AOI shapefile
    "data/processed/damaturu_100m_poly.geojson"  # output path
)

Values in clipped area before processing:
Unprocessed min in area: -99999.0
Unprocessed max in area: 1016.6382446289062
Density raster min: 0.4713766276836395
Density raster max: 99999.0

Created 9896 polygons
Value range: 0.47 to 99999.00


,geometry,population_density
0,"POLYGON ((11.98625 11.79292, 11.98625 11.79208...",0.763216
1,"POLYGON ((11.98708 11.79292, 11.98708 11.79208...",0.755720
2,"POLYGON ((11.98792 11.79292, 11.98792 11.79208...",0.744945
3,"POLYGON ((11.98875 11.79292, 11.98875 11.79208...",0.741613
4,"POLYGON ((11.98958 11.79292, 11.98958 11.79208...",0.722886
...,...,...
9891,"POLYGON ((11.94625 11.69958, 11.94625 11.69875...",1.546591
9892,"POLYGON ((11.94708 11.69958, 11.94708 11.69875...",1.541078
9893,"POLYGON ((11.94792 11.69958, 11.94792 11.69875...",1.591542
9894,"POLYGON ((11.94875 11.69958, 11.94875 11.69875...",1.583845


In [3]:
with rasterio.open("data/raw/nga_ppp_2020_UNadj.tif") as src:
    data = src.read(1)  # read first band
    print(f"Original raster min: {data.min()}")
    print(f"Original raster max: {data.max()}")

Original raster min: -99999.0
Original raster max: 1016.6382446289062


In [4]:
damaturu_pd100m_poly = gpd.read_file("data/processed/damaturu_pd100m_poly.geojson")
print(f"Min population: {damaturu_pd100m_poly['population_density'].min()}")
print(f"Max population: {damaturu_pd100m_poly['population_density'].max()}")

Min population: 0.4713766276836395
Max population: 32.23027801513672


# 2.0 Population Density (GCP Analysis)

In [8]:
# save Damaturu population density GCP tif as variable
damaturu_pd_gcp = rasterio.open("data/interim/2024-12-nigeria-damaturu_02-process-output_spatial_damaturu_population.tif")

# check coordinate system of damaturu_pd_gcp
print(f"coordinate system: {damaturu_pd_gcp.crs}")

coordinate system: EPSG:4326


In [10]:
# convert damaturu_pd_gcp from tif to GeoJSON for export and viz
damaturu_pd_gcp_geojson = raster_to_polygons(
    "data/interim/2024-12-nigeria-damaturu_02-process-output_spatial_damaturu_population.tif",
    "data/raw/AOI/damaturu.shp",
    "data/processed/damaturu_pd_gcp.geojson"
)


Created 4799 polygons
Value range: 3.46 to 71.60


# 3.0 IDP (Location, Location by Site Type, and Change in IDP Locations by Site Types (i.e, "IDP Growth"))

In [45]:
# read in "Latest Master List" Damaturu IDP R43 data
damaturu_idp_r43 = pd.read_excel("data/raw/DTM_Nigeria_R43_Master_List_IDP_Updated_0.xlsx")

In [ ]:
# read in "Latest Site Assessment" Damaturu IDP R47 data
damaturu_idp_r47 = pd.read_excel("data/raw/DTM_Nigeria_R47_Needs_Monitoring_North-east_public_web_v2.xlsx")

In [41]:
# gain insight into damaturu_idp_r47
print(damaturu_idp_r47.head())
print(damaturu_idp_r47.columns)

  DTM round Survey Date  KI's Population types Site accessibility  \
0       R47  2024-02-18     2       IDPs in HC                Yes   
1       R47  2024-02-18     1       IDPs in HC                Yes   
2       R47  2024-02-27     2        Returnees                Yes   
3       R47  2024-02-18     1       IDPs in HC                Yes   
4       R47  2024-02-18     2       IDPs in HC                Yes   

  Reasons for inaccessibility Site safe area New location Location type  \
0                         NaN            Yes           No          IDPs   
1                         NaN            Yes           No          IDPs   
2                         NaN            Yes           No       Returns   
3                         NaN            Yes           No          IDPs   
4                         NaN            Yes           No          IDPs   

     State  ... Main barriers                      Main barrier first  \
0  Adamawa  ...           NaN  Firewood collection points are

In [46]:
# gain insight into damaturu_idp_r43
print(damaturu_idp_r43.head())
print(damaturu_idp_r43.columns)

  Date of Survey                     Population type    State    LGA     Ward  \
0     2022-10-05  IDPs dispersed in Host Communities  ADAMAWA  Demsa    BILLE   
1     2022-10-05  IDPs dispersed in Host Communities  ADAMAWA  Demsa    BILLE   
2     2022-10-05  IDPs dispersed in Host Communities  ADAMAWA  Demsa  BORRONG   
3     2022-10-05  IDPs dispersed in Host Communities  ADAMAWA  Demsa  BORRONG   
4     2022-10-05  IDPs dispersed in Host Communities  ADAMAWA  Demsa  BORRONG   

      SSID  SITE NAME  latitude  longitude  Households  ...  \
0  AD_H001      Bille   9.26325   11.90344          80  ...   
1  AD_H002    Gejembo   9.26751   11.90238          32  ...   
2  AD_H004      Tikka   9.54568   12.23103          40  ...   
3  AD_H005  Borrong B   9.53868   12.17874         355  ...   
4  AD_H006  Borrong A   9.53608   12.18101         380  ...   

   Male Children 6-12 y Female Children 6-12 y Male Youth 13-17 y  \
0                  80.0                   80.0               57.0

In [47]:
# gain insight into damaturu_idp_r43
with pd.option_context('display.max_rows', 5, 'display.max_columns', None): 
    display(damaturu_idp_r43)

,Date of Survey,Population type,State,LGA,Ward,SSID,SITE NAME,latitude,longitude,Households,Individuals,Site start date,Site Classification,Reason of displacement of majority of IDPs,1.3.g.4 Settlement/site type,# HHs arrived to the location in 2022,# INDs arrived to the location in 2022,# HHs arrived to the location in 2021,# INDs arrived to the location in 2021,# HHs arrived to the location before 2021 (2014 - 2020),# INDs arrived to the location before 2021 (2014 - 2020),Male Infants 1 y,Female Infants 1 y,Male Children 1-5 y,Female Children 1-5 y,Male Children 6-12 y,Female Children 6-12 y,Male Youth 13-17 y,Female Youth 13-17 y,Male Adults 18-59 y,Female Adults 18-59 y,Male Elderly 60 y,Female Elderly 60 y,Total male,Total female
0,2022-10-05,IDPs dispersed in Host Communities,ADAMAWA,Demsa,BILLE,AD_H001,Bille,9.26325,11.90344,80,480,2017-12-04,rural,Communal Clash,In Host Community Families,4.0,24.0,7.0,42.0,69.0,414.0,14.0,15.0,26.0,27.0,80.0,80.0,57.0,57.0,43.0,43.0,19.0,19.0,239.0,241.0
1,2022-10-05,IDPs dispersed in Host Communities,ADAMAWA,Demsa,BILLE,AD_H002,Gejembo,9.26751,11.90238,32,192,2017-12-04,rural,Communal Clash,Existing Village,6.0,36.0,8.0,48.0,18.0,108.0,10.0,10.0,15.0,16.0,21.0,21.0,20.0,21.0,19.0,20.0,9.0,10.0,94.0,98.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2448,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40,185,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2449,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,483427,2375476,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
# gain insight into damaturu_idp_r47
with pd.option_context('display.max_rows', 5, 'display.max_columns', None): 
    display(damaturu_idp_r47)

,DTM round,Survey Date,KI's,Population types,Site accessibility,Reasons for inaccessibility,Site safe area,New location,Location type,State,State P-code,LGA,LGA P-code,Ward,Ward P-code,Site type,Site classification,SSID,Site location name,Site status,Hh,Ind,Site start date,Site closure,Site ecd,Site closing date,Site area measurement,Estimated site area,Site extension,Estimated site extension,Land ownership,Most common type of shelter,Type of shelter specify,Service provider,Ccm support,Ccm support organization,Registration activity,Registration activity type,Registration activity type/biometric registration1,Registration activity type/paper/manual registration,Reason for displacement,Primary origin state,Primary Origin LGA,Primary origin ward,Secondary origin state,Secondary Origin LGA,Secondary origin ward,Natural hazard risk,Male infants (<1y),Female infants (<1y),Male children (1-5y),Female children (1-5y),Male children (6-12y),Female children (6-12y),Male youth (13-17y),Female youth (13-17y),Male adults (18-59y),Female adults (18-59y),Male elderly (60 +y),Female elderly (60 +y),Total male,Total female,Pregnant women,Breastfeeding mothers,Elderly persons,Total number of individuals with other vulnerabilities,IDPs/Returnees-HHs Arrival- In 2024,IDPs/Returnees-INDs Arrival- In 2024,IDPs/Returnees-HHs Arrival- In 2023,IDPs/Returnees-INDs Arrival- In 2023,IDPs/Returnees-HHs Arrival- In 2022,IDPs/Returnees-INDs Arrival- In 2022,IDPs/Returnees-HHs Arrival- In 2021,IDPs/Returnees-INDs Arrival- In 2021,IDPs/Returnees-HHs Arrival- Before 2021 (2014-2020),IDPs/Returnees-INDs Arrival- Before 2021 (2014-2020),Displaced previously,Intention of majority,If no why,If yes when,If yes why,Area of intended return,State return intended,Lga return intended,Ward return intended,Returnee status,Initially displaced country,Initially displaced state,Initially displaced lga,Initially displaced ward,Majority year of returns (2024) - hh,Majority year of returns (2024) - ind,Majority year of returns (2023) - hh,Majority year of returns (2023) - ind,Majority year of returns (2022) - hh,Majority year of returns (2022) - ind,Majority year of returns (2021) - hh,Majority year of returns (2021) - ind,Majority year of returns (2014 - 2020) - hh,Majority year of returns (2014 - 2020) - ind,Reason for return,Living outside hh,Living outside,Living emergency shelter,Living makeshift shelter,Living indoors,Living pre exisiting shelter,Living traditinal shelter,Living safe cooking facilities,Shelter have locks,Reports and experiences of thefts,No damage houses hh,No damage houses ind,Paritally damaged houses hh,Paritally damaged houses ind,Fully damaged hh,Fully damaged ind,Most needed nfi,Second most needed nfi,Third most needed nfi,Truck accessible,Is there access to support,If yes how long ago did the site receive last assistance,Community access to information distribution,Main unfufilled need,Drainage on site,Need shelter material,Most needed shelter material,Second most needed shelter material,Third most needed shelter material,Site facilitation support,Cccm related activites,Location of sites main water source,Community feel safe wash,How operational are they,Average amount of water available daily,Are there complaints about drinking water quality,What are the main problem with the water,Sanitation facilities,Condition of most of the toilets,Number of toilets,Number of non functioning toilets on-site,Sanitation people,Availability of separate male and female toilets,Availability of separate bathing areas,Toilets/bathrooms have locks from the inside,Number of latrines that are lit,Main garbage disposal,Number of garbage disposal,Physical disabilities,Garbage and solid waste problem in site,Availability of hand-washing station filled in with water and soap close to the toilets,Hygiene awareness period,Female hygiene,Evidence of open defecation,Access to food support,If yes how long ago did the site receive last food assistance,Food provider,Access to funct

In [ ]:
# filter damaturu_idp_r47 to only include rows where the "SSID" column is "YB_H049", "YB_S018", "YB_S004", or "YB_S006"
damaturu_idp_r47_filtered = damaturu_idp_r47[damaturu_idp_r47['SSID'].isin(['YB_H049', 'YB_S018', 'YB_S004', 'YB_S006'])]

In [43]:
# gain insight into damaturu_idp_r47_filtered
with pd.option_context('display.max_rows', 5, 'display.max_columns', None): 
    display(damaturu_idp_r47_filtered)

,DTM round,Survey Date,KI's,Population types,Site accessibility,Reasons for inaccessibility,Site safe area,New location,Location type,State,State P-code,LGA,LGA P-code,Ward,Ward P-code,Site type,Site classification,SSID,Site location name,Site status,Hh,Ind,Site start date,Site closure,Site ecd,Site closing date,Site area measurement,Estimated site area,Site extension,Estimated site extension,Land ownership,Most common type of shelter,Type of shelter specify,Service provider,Ccm support,Ccm support organization,Registration activity,Registration activity type,Registration activity type/biometric registration1,Registration activity type/paper/manual registration,Reason for displacement,Primary origin state,Primary Origin LGA,Primary origin ward,Secondary origin state,Secondary Origin LGA,Secondary origin ward,Natural hazard risk,Male infants (<1y),Female infants (<1y),Male children (1-5y),Female children (1-5y),Male children (6-12y),Female children (6-12y),Male youth (13-17y),Female youth (13-17y),Male adults (18-59y),Female adults (18-59y),Male elderly (60 +y),Female elderly (60 +y),Total male,Total female,Pregnant women,Breastfeeding mothers,Elderly persons,Total number of individuals with other vulnerabilities,IDPs/Returnees-HHs Arrival- In 2024,IDPs/Returnees-INDs Arrival- In 2024,IDPs/Returnees-HHs Arrival- In 2023,IDPs/Returnees-INDs Arrival- In 2023,IDPs/Returnees-HHs Arrival- In 2022,IDPs/Returnees-INDs Arrival- In 2022,IDPs/Returnees-HHs Arrival- In 2021,IDPs/Returnees-INDs Arrival- In 2021,IDPs/Returnees-HHs Arrival- Before 2021 (2014-2020),IDPs/Returnees-INDs Arrival- Before 2021 (2014-2020),Displaced previously,Intention of majority,If no why,If yes when,If yes why,Area of intended return,State return intended,Lga return intended,Ward return intended,Returnee status,Initially displaced country,Initially displaced state,Initially displaced lga,Initially displaced ward,Majority year of returns (2024) - hh,Majority year of returns (2024) - ind,Majority year of returns (2023) - hh,Majority year of returns (2023) - ind,Majority year of returns (2022) - hh,Majority year of returns (2022) - ind,Majority year of returns (2021) - hh,Majority year of returns (2021) - ind,Majority year of returns (2014 - 2020) - hh,Majority year of returns (2014 - 2020) - ind,Reason for return,Living outside hh,Living outside,Living emergency shelter,Living makeshift shelter,Living indoors,Living pre exisiting shelter,Living traditinal shelter,Living safe cooking facilities,Shelter have locks,Reports and experiences of thefts,No damage houses hh,No damage houses ind,Paritally damaged houses hh,Paritally damaged houses ind,Fully damaged hh,Fully damaged ind,Most needed nfi,Second most needed nfi,Third most needed nfi,Truck accessible,Is there access to support,If yes how long ago did the site receive last assistance,Community access to information distribution,Main unfufilled need,Drainage on site,Need shelter material,Most needed shelter material,Second most needed shelter material,Third most needed shelter material,Site facilitation support,Cccm related activites,Location of sites main water source,Community feel safe wash,How operational are they,Average amount of water available daily,Are there complaints about drinking water quality,What are the main problem with the water,Sanitation facilities,Condition of most of the toilets,Number of toilets,Number of non functioning toilets on-site,Sanitation people,Availability of separate male and female toilets,Availability of separate bathing areas,Toilets/bathrooms have locks from the inside,Number of latrines that are lit,Main garbage disposal,Number of garbage disposal,Physical disabilities,Garbage and solid waste problem in site,Availability of hand-washing station filled in with water and soap close to the toilets,Hygiene awareness period,Female hygiene,Evidence of open defecation,Access to food support,If yes how long ago did the site receive last food assistance,Food provider,Access to funct

## 3.1 KMZ to Shapefile Conversion

In [65]:
# convert IDP location KMZ to shapefile
idp_r39_shapefile = kmz_to_shapefile(
    "data/raw/Round_39_North_East_IDP_Camp_Sites_KMZ.kmz",
    "data/interim/idp_r39_shapefile.shp"
)

/Users/carolinecullinan/dev/wb/damaturu-nigeria-idp/src/idp.py:31: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(output_shp)


## 3.2 IDP Damaturu Clip

In [66]:
# check coordinate system of IDP shapefile
print(f"coordinate system: {idp_r39_shapefile.crs}")

coordinate system: EPSG:4326


In [67]:
# clip IDP shapefile to Damaturu shapefile
damaturu_idp_r39 = gpd.clip(idp_r39_shapefile, aoi)

In [68]:
# save damaturu_idp_r39 to shapefile
damaturu_idp_r39.to_file("data/interim/damaturu_idp_r39.shp")

/var/folders/xy/9wgvl7914cj_8zq0xmgxg4hw0000gn/T/ipykernel_39648/716237603.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  damaturu_idp_r39.to_file("data/interim/damaturu_idp_r39.shp")


### 3.3 IDP Damaturu Exploration

In [69]:
# map and explore damaturu_idp_r39
damaturu_idp_r39.explore()

In [70]:
# gain insight into damaturu_idp_r39
print(damaturu_idp_r39.head())
print(damaturu_idp_r39.columns)
print(damaturu_idp_r39[['Description']])


                  Name                                        Description  \
293   MOH'D GOMBE FARM  <html xmlns:fo="http://www.w3.org/1999/XSL/For...   
289   BUKAR ABBA HOUSE  <html xmlns:fo="http://www.w3.org/1999/XSL/For...   
288        ABARI Y.B.C  <html xmlns:fo="http://www.w3.org/1999/XSL/For...   
292  HURI VILLAGE CAMP  <html xmlns:fo="http://www.w3.org/1999/XSL/For...   

                                geometry  
293  POINT Z (12.03584 11.72675 0.00000)  
289  POINT Z (11.98183 11.72985 0.00000)  
288  POINT Z (11.98050 11.72993 0.00000)  
292  POINT Z (12.01215 11.73905 0.00000)  
Index(['Name', 'Description', 'geometry'], dtype='object')
                                           Description
293  <html xmlns:fo="http://www.w3.org/1999/XSL/For...
289  <html xmlns:fo="http://www.w3.org/1999/XSL/For...
288  <html xmlns:fo="http://www.w3.org/1999/XSL/For...
292  <html xmlns:fo="http://www.w3.org/1999/XSL/For...


In [71]:
# damaturu_idp_r39 shapefile to dataframe for easy data analysis
damaturu_idp_r39_df = transform_idp_description(damaturu_idp_r39)

In [10]:
# save damaturu_idp_r39_df to csv
damaturu_idp_r39_df.to_csv("data/interim/damaturu_idp_r39_df.csv", index=False)

In [72]:
# damaturu_idp_r39 shapefile to clean geojson with all attributes as columns for easy mapping
damaturu_idp_r39_clean = transform_idp_complete(damaturu_idp_r39)

In [73]:
# gain insight into damaturu_idp_r39_clean and look at what columns we have
print("Columns in transformed data:")
print(damaturu_idp_r39_clean.columns)

# gain insight into damaturu_idp_r39_clean and look at first row
print("\nFirst site data:")
print(damaturu_idp_r39_clean.iloc[0])

Columns in transformed data:
Index(['name', 'dtm_round', 'date', 'state', 'state_pcode', 'lga', 'ward',
       'siteid', 'site_name', 'longitude',
       ...
       'percent_of_children_attending_school',
       'occupation_of_majority_of_idps', '#_of_hh_with_source_of_income',
       'access_to_income_generating_activities', 'livestock_on_site',
       'idps_have_access_to_land_for_cultivation',
       'most_trusted_source_of_info',
       'main_topic_community_is_requesting_info',
       'serious_problem_due_to_lack_information', 'geometry'],
      dtype='object', length=102)

First site data:
name                                                     MOH'D GOMBE FARM
dtm_round                                                             R39
date                                                           06/10/2021
state                                                                YOBE
state_pcode                                                         NG036
                           

In [74]:
# gain insight into damaturu_idp_r39_clean 
with pd.option_context('display.max_rows', 5, 'display.max_columns', None): 
    display(damaturu_idp_r39_clean)

,name,dtm_round,date,state,state_pcode,lga,ward,siteid,site_name,longitude,latitude,site_status,site_type,site_classification,approximate_site_area,site_management_agency,sma_type,registration_activity,wash_support,cccm_support,cccm_organization,health_support,shelter_support,nfi_support,food_support,protection_support,education_support,livelihood_support,pss_support,pss_organization,land_ownership,most_common_type_of_shelter,number_of_households,number_of_individuals,boys_<1,girls_<1,boys_1___5_y,girls_1___5_y,boys_6to12y,girls_6to12y,boys_13to17y,girls_13to17y,men_18___59_y,women_18___59_y,elderly_men_60+_y,elderly_women_60+_y,have_idps_been_displaced_previously,reason_for_displacement_of_majority,natural_hazard_risks,intended_area_of_return,reasons_to_return_home,reasons_not_to_return_home,percent_hh_living_outside_no_shelter,percent_hh_living_in_ems,percent_hh_living_in_makeshift_self_made_shelter,percent_hh_living_indoors_solid_walls,percent_hh_have_access_to_electricity,percent_hh_have_access_to_safe_cooking_facilities,percent_hh_have_private_living_area,percent_hh_have_mosquito_nets,most_needed_nfi,most_unmet_needs,location_of_site_main_water_source,main_drinking_water_source,avg_amount_of_water_available__day_person,is_drinking_water_potable,complaints_about_drinking_water_quality,condition_of_most_of_the_toilets,#_of_functioning_toilets,main_garbage_disposal,garbage_and_solid_waste_problem,availability_of_hand_washing_station,hygiene_promotion_campaign,evidence_of_open_defecation,access_to_food,access_to_market_near_from_the_site,frequency_of_food_distribution,most_common_source_for_obtaining_food,screening_for_malnutrition,most_health_problem,regular_access_to_medicine,access_to_health_facility,location_of_health_facilities,main_provider_of_health_facilities,main_provider_of_health_services,primary_health_care_for_children,primary_health_care_for_adult,secondary_health_care_for_children,secondary_health_care_for_adult,traditional_health_care,access_to_education_services,location_of_education_facilities,percent_of_children_attending_school,occupation_of_majority_of_idps,#_of_hh_with_source_of_income,access_to_income_generating_activities,livestock_on_site,idps_have_access_to_land_for_cultivation,most_trusted_source_of_info,main_topic_community_is_requesting_info,serious_problem_due_to_lack_information,geometry
0,MOH'D GOMBE FARM,R39,06/10/2021,YOBE,NG036,DAMATURU,MAISANDARI/WAZIRI IBRAHIM ESTATE,YB_S004,MOH'D GOMBE FARM,12.03584,11.72675,informal,Collective Settlement/Centre,Spontaneous,"183,444 sq.m",False,none,True,True,False,No CCCM Organization,False,True,False,True,False,True,False,False,No PSS ORGANIZATION,private building,individual house,55.0,330.0,8.0,10.0,11.0,15.0,16.0,20.0,39.0,46.0,63.0,88.0,6.0,8.0,False,insurgency,none,place of origin,improved security,I want to go back,none,none,none,>75%,none,25%_50%,none,51%_75%,kitchen sets,Food,on-site (<10 minutes),piped water supply,unknown,True,False,not so good (not hygienic),25.0,garbage pit,False,none,False,False,yes onsite,True,once_a_month,cultivated,False,malaria,False,False,off-site (> 3 km),government,government,False,True,False,False,False,True,onsite,<25%,agro-pastoralism,0.0,False,True,True,government official,safety and security,True,POINT Z (12.03584 11.72675 0.00000)
1,BUKAR ABBA HOUSE,R39,08/10/2021,YOBE,NG036,DAMATURU,DAMATURU CENTRAL,YB_S021,BUKAR ABBA HOUSE,11.98183,11.72985,informal,Collective Settlement/Centre,Spontaneous,"65,276 sq.m",False,none,True,True,False,No CCCM Organization,True,True,True,False,False,True,True,False,No PSS ORGANIZATION,Public/Government,individual house,78.0,468.0,19.0,25.0,25.0,27.0,28.0,31.0,23.0,27.0,104.0,149.0,4.0,6.0,False,insurgency,none,none stay here,improved security,I want to go back,none,none,none,>75%,25%_50%,51%_75%,<25%,51%_75%,mattress,Food,on-site (<10 minutes),piped water supply,unknown,True,False,not so good (not hygienic),5.0,garbage pit,False,none,False,False,False,False,never,cash (personal mone

In [75]:
# add column "population_type" to damaturu_idp_r39_clean that indicates whether IDPs are in "host community" or "IDP camp", 
# were "site_name", "BUKAR ABBA HOUSE" is "host community" and all other sites are "IDP camp"
damaturu_idp_r39_clean['population_type'] = damaturu_idp_r39_clean['site_name'].apply(lambda x: 'host community' if x == 'BUKAR ABBA HOUSE' else 'IDP camp')

In [76]:
# gain insight into damaturu_idp_r47_filtered
with pd.option_context('display.max_rows', 5, 'display.max_columns', None): 
    display(damaturu_idp_r47_filtered)

,DTM round,Survey Date,KI's,Population types,Site accessibility,Reasons for inaccessibility,Site safe area,New location,Location type,State,State P-code,LGA,LGA P-code,Ward,Ward P-code,Site type,Site classification,SSID,Site location name,Site status,Hh,Ind,Site start date,Site closure,Site ecd,Site closing date,Site area measurement,Estimated site area,Site extension,Estimated site extension,Land ownership,Most common type of shelter,Type of shelter specify,Service provider,Ccm support,Ccm support organization,Registration activity,Registration activity type,Registration activity type/biometric registration1,Registration activity type/paper/manual registration,Reason for displacement,Primary origin state,Primary Origin LGA,Primary origin ward,Secondary origin state,Secondary Origin LGA,Secondary origin ward,Natural hazard risk,Male infants (<1y),Female infants (<1y),Male children (1-5y),Female children (1-5y),Male children (6-12y),Female children (6-12y),Male youth (13-17y),Female youth (13-17y),Male adults (18-59y),Female adults (18-59y),Male elderly (60 +y),Female elderly (60 +y),Total male,Total female,Pregnant women,Breastfeeding mothers,Elderly persons,Total number of individuals with other vulnerabilities,IDPs/Returnees-HHs Arrival- In 2024,IDPs/Returnees-INDs Arrival- In 2024,IDPs/Returnees-HHs Arrival- In 2023,IDPs/Returnees-INDs Arrival- In 2023,IDPs/Returnees-HHs Arrival- In 2022,IDPs/Returnees-INDs Arrival- In 2022,IDPs/Returnees-HHs Arrival- In 2021,IDPs/Returnees-INDs Arrival- In 2021,IDPs/Returnees-HHs Arrival- Before 2021 (2014-2020),IDPs/Returnees-INDs Arrival- Before 2021 (2014-2020),Displaced previously,Intention of majority,If no why,If yes when,If yes why,Area of intended return,State return intended,Lga return intended,Ward return intended,Returnee status,Initially displaced country,Initially displaced state,Initially displaced lga,Initially displaced ward,Majority year of returns (2024) - hh,Majority year of returns (2024) - ind,Majority year of returns (2023) - hh,Majority year of returns (2023) - ind,Majority year of returns (2022) - hh,Majority year of returns (2022) - ind,Majority year of returns (2021) - hh,Majority year of returns (2021) - ind,Majority year of returns (2014 - 2020) - hh,Majority year of returns (2014 - 2020) - ind,Reason for return,Living outside hh,Living outside,Living emergency shelter,Living makeshift shelter,Living indoors,Living pre exisiting shelter,Living traditinal shelter,Living safe cooking facilities,Shelter have locks,Reports and experiences of thefts,No damage houses hh,No damage houses ind,Paritally damaged houses hh,Paritally damaged houses ind,Fully damaged hh,Fully damaged ind,Most needed nfi,Second most needed nfi,Third most needed nfi,Truck accessible,Is there access to support,If yes how long ago did the site receive last assistance,Community access to information distribution,Main unfufilled need,Drainage on site,Need shelter material,Most needed shelter material,Second most needed shelter material,Third most needed shelter material,Site facilitation support,Cccm related activites,Location of sites main water source,Community feel safe wash,How operational are they,Average amount of water available daily,Are there complaints about drinking water quality,What are the main problem with the water,Sanitation facilities,Condition of most of the toilets,Number of toilets,Number of non functioning toilets on-site,Sanitation people,Availability of separate male and female toilets,Availability of separate bathing areas,Toilets/bathrooms have locks from the inside,Number of latrines that are lit,Main garbage disposal,Number of garbage disposal,Physical disabilities,Garbage and solid waste problem in site,Availability of hand-washing station filled in with water and soap close to the toilets,Hygiene awareness period,Female hygiene,Evidence of open defecation,Access to food support,If yes how long ago did the site receive last food assistance,Food provider,Access to funct

In [77]:
# add a new column, "Sanitation facilities", to damaturu_idp_r39_clean where the value for "site_name" "BUKAR ABBA HOUSE" is "Flush to pit latrine", 
    ## the value for "site_name" "ABARI Y.B.C" is "Flush to pit latrine", the value for for "site_name" "MOH'D GOMBE FARM" is "Pit latrine with slab", and
    ## the value for "site_name" "HURI VILLAGE CAMP" is "Pit latrine with slab"
damaturu_idp_r39_clean['sanitation_facilities'] = damaturu_idp_r39_clean['site_name'].apply(lambda x: 'Flush to pit latrine' if x == 'BUKAR ABBA HOUSE' or x == 'ABARI Y.B.C' else 'Pit latrine with slab')

In [78]:
# add a new column, "Drainage on site", to damaturu_idp_r39_clean where the value for "site_name" "BUKAR ABBA HOUSE" is "Yes" and all other "site_name" values are "No"
damaturu_idp_r39_clean['drainage_on_site'] = damaturu_idp_r39_clean['site_name'].apply(lambda x: 'Yes' if x == 'BUKAR ABBA HOUSE' else 'No')

In [79]:
# add a new column, "Average amount of water available daily", to damaturu_idp_r39_clean where the value for all rows is "Unknown"
damaturu_idp_r39_clean['average_amount_of_water_available daily'] = 'Unknown'

In [80]:
# gain insight into damaturu_idp_r39_clean 
with pd.option_context('display.max_rows', 5, 'display.max_columns', None): 
    display(damaturu_idp_r39_clean)

,name,dtm_round,date,state,state_pcode,lga,ward,siteid,site_name,longitude,latitude,site_status,site_type,site_classification,approximate_site_area,site_management_agency,sma_type,registration_activity,wash_support,cccm_support,cccm_organization,health_support,shelter_support,nfi_support,food_support,protection_support,education_support,livelihood_support,pss_support,pss_organization,land_ownership,most_common_type_of_shelter,number_of_households,number_of_individuals,boys_<1,girls_<1,boys_1___5_y,girls_1___5_y,boys_6to12y,girls_6to12y,boys_13to17y,girls_13to17y,men_18___59_y,women_18___59_y,elderly_men_60+_y,elderly_women_60+_y,have_idps_been_displaced_previously,reason_for_displacement_of_majority,natural_hazard_risks,intended_area_of_return,reasons_to_return_home,reasons_not_to_return_home,percent_hh_living_outside_no_shelter,percent_hh_living_in_ems,percent_hh_living_in_makeshift_self_made_shelter,percent_hh_living_indoors_solid_walls,percent_hh_have_access_to_electricity,percent_hh_have_access_to_safe_cooking_facilities,percent_hh_have_private_living_area,percent_hh_have_mosquito_nets,most_needed_nfi,most_unmet_needs,location_of_site_main_water_source,main_drinking_water_source,avg_amount_of_water_available__day_person,is_drinking_water_potable,complaints_about_drinking_water_quality,condition_of_most_of_the_toilets,#_of_functioning_toilets,main_garbage_disposal,garbage_and_solid_waste_problem,availability_of_hand_washing_station,hygiene_promotion_campaign,evidence_of_open_defecation,access_to_food,access_to_market_near_from_the_site,frequency_of_food_distribution,most_common_source_for_obtaining_food,screening_for_malnutrition,most_health_problem,regular_access_to_medicine,access_to_health_facility,location_of_health_facilities,main_provider_of_health_facilities,main_provider_of_health_services,primary_health_care_for_children,primary_health_care_for_adult,secondary_health_care_for_children,secondary_health_care_for_adult,traditional_health_care,access_to_education_services,location_of_education_facilities,percent_of_children_attending_school,occupation_of_majority_of_idps,#_of_hh_with_source_of_income,access_to_income_generating_activities,livestock_on_site,idps_have_access_to_land_for_cultivation,most_trusted_source_of_info,main_topic_community_is_requesting_info,serious_problem_due_to_lack_information,geometry,population_type,sanitation_facilities,drainage_on_site,average_amount_of_water_available daily
0,MOH'D GOMBE FARM,R39,06/10/2021,YOBE,NG036,DAMATURU,MAISANDARI/WAZIRI IBRAHIM ESTATE,YB_S004,MOH'D GOMBE FARM,12.03584,11.72675,informal,Collective Settlement/Centre,Spontaneous,"183,444 sq.m",False,none,True,True,False,No CCCM Organization,False,True,False,True,False,True,False,False,No PSS ORGANIZATION,private building,individual house,55.0,330.0,8.0,10.0,11.0,15.0,16.0,20.0,39.0,46.0,63.0,88.0,6.0,8.0,False,insurgency,none,place of origin,improved security,I want to go back,none,none,none,>75%,none,25%_50%,none,51%_75%,kitchen sets,Food,on-site (<10 minutes),piped water supply,unknown,True,False,not so good (not hygienic),25.0,garbage pit,False,none,False,False,yes onsite,True,once_a_month,cultivated,False,malaria,False,False,off-site (> 3 km),government,government,False,True,False,False,False,True,onsite,<25%,agro-pastoralism,0.0,False,True,True,government official,safety and security,True,POINT Z (12.03584 11.72675 0.00000),IDP camp,Pit latrine with slab,No,Unknown
1,BUKAR ABBA HOUSE,R39,08/10/2021,YOBE,NG036,DAMATURU,DAMATURU CENTRAL,YB_S021,BUKAR ABBA HOUSE,11.98183,11.72985,informal,Collective Settlement/Centre,Spontaneous,"65,276 sq.m",False,none,True,True,False,No CCCM Organization,True,True,True,False,False,True,True,False,No PSS ORGANIZATION,Public/Government,individual house,78.0,468.0,19.0,25.0,25.0,27.0,28.0,31.0,23.0,27.0,104.0,149.0,4.0,6.0,False,insurgency,none,none stay here,improved security,I want to go back,none,none,none,>75%,25%_50%,51%_75%,<25%,51%_75%,mattress,Food,on-site (<10 minutes),pipe

In [81]:
# export damaturu_idp_r39_clean to csv
damaturu_idp_r39_clean.to_csv("data/processed/damaturu_idp_r39_clean.csv", index=False)

In [82]:
# save damaturu_idp_r39_clean to geojson for mapping
damaturu_idp_r39_clean.to_file('data/processed/damaturu_idp_r39_clean.geojson', driver='GeoJSON')

# 4.0 Land Cover

In [11]:
# save Damaturu land GCP tif as variable
damaturu_lc = rasterio.open("data/interim/2024-12-nigeria-damaturu_02-process-output_spatial_damaturu_lc.tif")

# check coordinate system of damaturu_lc
print(f"coordinate system: {damaturu_pd_gcp.crs}")

coordinate system: EPSG:4326


In [12]:
# convert damaturu_lc from tif to GeoJSON for export and viz
damaturu_lc_geojson = raster_to_polygons(
    "data/interim/2024-12-nigeria-damaturu_02-process-output_spatial_damaturu_lc.tif",
    "data/raw/AOI/damaturu.shp",
    "data/processed/damaturu_lc.geojson"
)


Created 7429 polygons
Value range: 10.00 to 90.00


# 5.0 Built-Up Area Change

In [14]:
# save Damaturu built-up area wsf GCP tif as variable
damaturu_wsf = rasterio.open("data/interim/2024-12-nigeria-damaturu_02-process-output_spatial_damaturu_wsf_evolution_3857.tif")

# check coordinate system of damaturu_wsf
print(f"coordinate system: {damaturu_wsf.crs}") # crs is Web Mercator (EPSG:3857)

coordinate system: EPSG:3857


In [16]:
# check coordinate system of aoi, damaturu.shp
aoi = gpd.read_file("data/raw/AOI/damaturu.shp")
print(f"coordinate system: {aoi.crs}")

coordinate system: EPSG:4326


In [19]:
# convert aoi to Web Mercator (EPSG:3857)
aoi_wsf = aoi.to_crs(epsg=3857)

# save aoi_wsf to shapefile
aoi_wsf.to_file("data/interim/aoi_wsf.shp")


In [20]:
# convert damaturu_wsf from tif to GeoJSON for export and viz
damaturu_wsf_geojson = raster_to_polygons(
    "data/interim/2024-12-nigeria-damaturu_02-process-output_spatial_damaturu_wsf_evolution_3857.tif",
    "data/interim/aoi_wsf.shp",
    "data/processed/damaturu_wsf.geojson"
)


Created 11022 polygons
Value range: 1985.00 to 2015.00


### 5.1 Built-Up Area WSF 2019

In [43]:
# save Damaturu 2019 built-up area wsf GCP tif as variable
damaturu_wsf_2019 = rasterio.open("data/interim/WSF_2019.tif")

# check coordinate system of damaturu_wsf_2019
print(f"coordinate system: {damaturu_wsf_2019.crs}") # crs is EPSG:4326)

coordinate system: EPSG:4326


In [4]:
# convert damaturu_wsf_2019 from tif to GeoJSON for export and viz
damaturu_wsf_2019_geojson = raster_to_polygons(
    "data/interim/WSF_2019.tif",
    "data/raw/AOI/damaturu.shp",
    "data/processed/damaturu_wsf_2019.geojson"
)

Original data type: float32


# 6.0 Roads

In [6]:
# save Damaturu roads GCP .gpkg as variable
damaturu_roads = gpd.read_file("data/interim/2024-12-nigeria-damaturu_02-process-output_spatial_damaturu_osm_roads.gpkg")

In [7]:
# gain insight into damaturu_roads
print(damaturu_roads.head())
print(damaturu_roads.columns)

    length       time   mode  \
0   40.693   7.324740  drive   
1  179.171  16.125390  drive   
2   10.702   1.100777  drive   
3  124.743  11.226870  drive   
4  190.448  17.140320  drive   

                                            geometry  
0  LINESTRING (12.00817 11.73678, 12.00796 11.73648)  
1  LINESTRING (12.00817 11.73678, 12.00847 11.736...  
2  LINESTRING (12.00956 11.73591, 12.00961 11.73600)  
3  LINESTRING (12.00956 11.73591, 12.00996 11.735...  
4  LINESTRING (11.99645 11.74261, 11.99715 11.742...  
Index(['length', 'time', 'mode', 'geometry'], dtype='object')


In [8]:
# gain insight into damaturu_roads
with pd.option_context('display.max_rows', 5, 'display.max_columns', None): 
    display(damaturu_roads)

,length,time,mode,geometry
0,40.693,7.32474,drive,"LINESTRING (12.00817 11.73678, 12.00796 11.73648)"
1,179.171,16.12539,drive,"LINESTRING (12.00817 11.73678, 12.00847 11.736..."
...,...,...,...,...
13341,69.088,12.43584,drive,"LINESTRING (11.94926 11.71477, 11.94865 11.71461)"
13342,448.586,80.74548,drive,"LINESTRING (11.94463 11.71374, 11.94630 11.714..."


In [9]:
# save damaturu_roads to geojson for export and viz
damaturu_roads.to_file("data/processed/damaturu_roads.geojson", driver='GeoJSON')

# 7.0 Nighttime Lights

### 7.1 Average Nighttime Light Radiance


In [9]:
# save Damaturu nighttime light GCP tif as variable
damaturu_ntl = rasterio.open("data/interim/2024-12-nigeria-damaturu_02-process-output_spatial_damaturu_avg_rad_sum.tif")

# check coordinate system of damaturu_ntl
print(f"coordinate system: {damaturu_ntl.crs}")

coordinate system: EPSG:4326


In [8]:
# check coordinate system of aoi, damaturu.shp
aoi = gpd.read_file("data/raw/AOI/damaturu.shp")
print(f"coordinate system: {aoi.crs}")

coordinate system: EPSG:4326


In [11]:
# convert damaturu_ntl from tif to GeoJSON for export and viz
damaturu_ntl_geojson = raster_to_polygons(
    "data/interim/2024-12-nigeria-damaturu_02-process-output_spatial_damaturu_avg_rad_sum.tif",
    "data/raw/AOI/damaturu.shp",
    "data/processed/damaturu_ntl.geojson"
)

Original data type: float64


### 7.2 Rate of Change in Nighttime Light Radiance

In [5]:
# gain more insight into damaturu nighttime light change GCP tif data

# read the tif file
with rasterio.open("data/interim/2024-12-nigeria-damaturu_02-process-output_spatial_damaturu_linfit.tif") as src:
   
    # read data as a numpy array
    data = src.read(1)  # read first band
    
    # get basic stats
    min_val = np.nanmin(data)
    max_val = np.nanmax(data)
    mean_val = np.nanmean(data)
    median_val = np.nanmedian(data)
    
    # print stats
    print(f"Min value: {min_val}")
    print(f"Max value: {max_val}")
    print(f"Mean value: {mean_val}")
    print(f"Median value: {median_val}")
    
    # get a histogram of values to understand distribution
    # ignore no-data values (usually very negative numbers or NaN)
    valid_data = data[data > -9999]  # adjust this threshold based on your no-data value
    
    # print distribution information
    print("\nValue distribution:")
    print(f"25th percentile: {np.percentile(valid_data, 25)}")
    print(f"75th percentile: {np.percentile(valid_data, 75)}")
    
    # print first few non-zero values to see actual data points
    non_zero = valid_data[valid_data > 0]
    print("\nSample of non-zero values:")
    print(non_zero[:10])

Min value: -9999.0
Max value: 1.9825619832589798
Mean value: -4545.928598613609
Median value: 0.05359354301797102

Value distribution:
25th percentile: 0.08934509403857702
75th percentile: 0.47733468589204925

Sample of non-zero values:
[0.04223579 0.04208608 0.05160682 0.0494282  0.05032423 0.04646537
 0.04464805 0.06632265 0.06031924 0.05770802]


In [6]:
# save Damaturu nighttime light change GCP tif as variable
damaturu_ntl_change = rasterio.open("data/interim/2024-12-nigeria-damaturu_02-process-output_spatial_damaturu_linfit.tif")

# check coordinate system of damaturu_ntl_change
print(f"coordinate system: {damaturu_ntl_change.crs}")

coordinate system: EPSG:4326


In [7]:
# convert damaturu_ntl_change from tif to GeoJSON for export and viz
damaturu_ntl_change_geojson = raster_to_polygons(
    "data/interim/2024-12-nigeria-damaturu_02-process-output_spatial_damaturu_linfit.tif",
    "data/raw/AOI/damaturu.shp",
    "data/processed/damaturu_ntl_change.geojson"
)

Original data type: float64


# 8.0 WASH

WASH Data [Institute from Health Metrics and Evaluation]: https://cloud.ihme.washington.edu/s/bkH2X2tFQMejMxy?path=%2F

## 8.1 Services - Access to Sanitation (S_IMP - access to any improved sanitation)


### 2009


In [13]:
# save global mean 2009 S_IMP WASH tif as variable
global_2009_s_imp = rasterio.open("data/raw/IHME_LMIC_WASH_2000_2017_S_IMP_PERCENT_MEAN_2009_Y2020M06D02.tif")

# check coordinate system of global_2009_s_imp
print(f"coordinate system: {global_2009_s_imp.crs}") # crs is EPSG:4326

coordinate system: EPSG:4326


In [14]:
# clip global_2009_s_imp to Damaturu, Nigeria and convert from tif to GeoJSON for export and viz
damaturu_2009_s_imp_geojson = raster_to_polygons(
    "data/raw/IHME_LMIC_WASH_2000_2017_S_IMP_PERCENT_MEAN_2009_Y2020M06D02.tif",
    "data/raw/AOI/damaturu.shp",
    "data/processed/damaturu_2009_s_imp.geojson"
)

Original data type: float32


### 2017


In [15]:
# save global mean 2017 S_IMP WASH tif as variable
global_2017_s_imp = rasterio.open("data/raw/IHME_LMIC_WASH_2000_2017_S_IMP_PERCENT_MEAN_2017_Y2020M06D02.tif")

# check coordinate system of global_2017_s_imp
print(f"coordinate system: {global_2017_s_imp.crs}") # crs is EPSG:4326

coordinate system: EPSG:4326


In [16]:
# clip global_2017_s_imp to Damaturu, Nigeria and convert from tif to GeoJSON for export and viz
damaturu_2017_s_imp_geojson = raster_to_polygons(
    "data/raw/IHME_LMIC_WASH_2000_2017_S_IMP_PERCENT_MEAN_2017_Y2020M06D02.tif",
    "data/raw/AOI/damaturu.shp",
    "data/processed/damaturu_2017_s_imp.geojson"
)

Original data type: float32


## 8.2 Services - Access to Drainage (S_PIPED - access to sewer and sanitation facilities)

### 2009

In [17]:
# save global mean 2009 S_PIPED WASH tif as variable
global_2009_s_piped = rasterio.open("data/raw/IHME_LMIC_WASH_2000_2017_S_PIPED_PERCENT_MEAN_2009_Y2020M06D02.tif")

# check coordinate system of global_2009_s_piped
print(f"coordinate system: {global_2009_s_piped.crs}") # crs is EPSG:4326

coordinate system: EPSG:4326


In [18]:
# clip global_2009_s_piped to Damaturu, Nigeria and convert from tif to GeoJSON for export and viz
damaturu_2009_s_piped_geojson = raster_to_polygons(
    "data/raw/IHME_LMIC_WASH_2000_2017_S_PIPED_PERCENT_MEAN_2009_Y2020M06D02.tif",
    "data/raw/AOI/damaturu.shp",
    "data/processed/damaturu_2009_s_piped.geojson"
)

Original data type: float32


### 2017


In [19]:
# save global mean 2017 S_PIPED WASH tif as variable
global_2017_s_piped = rasterio.open("data/raw/IHME_LMIC_WASH_2000_2017_S_PIPED_PERCENT_MEAN_2017_Y2020M06D02.tif")

# check coordinate system of global_2017_s_piped
print(f"coordinate system: {global_2017_s_piped.crs}") # crs is EPSG:4326

coordinate system: EPSG:4326


In [20]:
# clip global_2017_s_piped to Damaturu, Nigeria and convert from tif to GeoJSON for export and viz
damaturu_2017_s_piped_geojson = raster_to_polygons(
    "data/raw/IHME_LMIC_WASH_2000_2017_S_PIPED_PERCENT_MEAN_2017_Y2020M06D02.tif",
    "data/raw/AOI/damaturu.shp",
    "data/processed/damaturu_2017_s_piped.geojson"
)

Original data type: float32


## 8.3 Services -Access to Water Supply (W_PIPED - access to piped water)


### 2009

In [21]:
# save global mean 2009 W_PIPED WASH tif as variable
global_2009_w_piped = rasterio.open("data/raw/IHME_LMIC_WASH_2000_2017_W_PIPED_PERCENT_MEAN_2009_Y2020M06D02.tif")

# check coordinate system of global_2009_w_piped
print(f"coordinate system: {global_2009_w_piped.crs}") # crs is EPSG:4326

coordinate system: EPSG:4326


In [22]:
# clip global_2009_w_piped to Damaturu, Nigeria and convert from tif to GeoJSON for export and viz
damaturu_2009_w_piped_geojson = raster_to_polygons(
    "data/raw/IHME_LMIC_WASH_2000_2017_W_PIPED_PERCENT_MEAN_2009_Y2020M06D02.tif",
    "data/raw/AOI/damaturu.shp",
    "data/processed/damaturu_2009_w_piped.geojson"
)

Original data type: float32


### 2017


In [23]:
# save global mean 2017 W_PIPED WASH tif as variable
global_2017_w_piped = rasterio.open("data/raw/IHME_LMIC_WASH_2000_2017_W_PIPED_PERCENT_MEAN_2017_Y2020M06D02.tif")

# check coordinate system of global_2017_w_piped
print(f"coordinate system: {global_2017_w_piped.crs}") # crs is EPSG:4326

coordinate system: EPSG:4326


In [24]:
# clip global_2017_w_piped to Damaturu, Nigeria and convert from tif to GeoJSON for export and viz
damaturu_2017_w_piped_geojson = raster_to_polygons(
    "data/raw/IHME_LMIC_WASH_2000_2017_W_PIPED_PERCENT_MEAN_2017_Y2020M06D02.tif",
    "data/raw/AOI/damaturu.shp",
    "data/processed/damaturu_2017_w_piped.geojson"
)

Original data type: float32


# 9.0 Flooding

## 9.1 Pluvial Flooding


In [15]:
# save Damaturu pluvial flooding GCP tif as variable
damaturu_pf = rasterio.open("data/interim/2024-12-nigeria-damaturu_02-process-output_spatial_damaturu_pluvial_2020.tif")

# check coordinate system of damaturu_pf
print(f"coordinate system: {damaturu_pf.crs}")

coordinate system: EPSG:4326


In [16]:
# check coordinate system of aoi, damaturu.shp
aoi = gpd.read_file("data/raw/AOI/damaturu.shp")
print(f"coordinate system: {aoi.crs}")

coordinate system: EPSG:4326


In [18]:
# convert damaturu_pf from tif to GeoJSON for export and viz
damaturu_pf_geojson = raster_to_polygons(
    "data/interim/2024-12-nigeria-damaturu_02-process-output_spatial_damaturu_pluvial_2020.tif",
    "data/raw/AOI/damaturu.shp",
    "data/processed/damaturu_pf.geojson")

Original data type: uint8


## 9.2 Fluvial Flooding

In [19]:
# save Damaturu fluvial flooding GCP tif as variable
damaturu_ff = rasterio.open("data/interim/2024-12-nigeria-damaturu_02-process-output_spatial_damaturu_fluvial_2020.tif")

# check coordinate system of damaturu_ff
print(f"coordinate system: {damaturu_ff.crs}")

coordinate system: EPSG:4326


In [20]:
# check coordinate system of aoi, damaturu.shp
aoi = gpd.read_file("data/raw/AOI/damaturu.shp")
print(f"coordinate system: {aoi.crs}")

coordinate system: EPSG:4326


In [21]:
# convert damaturu_ff from tif to GeoJSON for export and viz
damaturu_ff_geojson = raster_to_polygons(
    "data/interim/2024-12-nigeria-damaturu_02-process-output_spatial_damaturu_fluvial_2020.tif",
    "data/raw/AOI/damaturu.shp",
    "data/processed/damaturu_ff.geojson")

Original data type: uint8


## 9.3 Combined Pluvial & Fluvial Flooding

In [22]:
# save Damaturu combined pluvial & fluvial flooding GCP tif as variable
damaturu_cf = rasterio.open("data/interim/2024-12-nigeria-damaturu_02-process-output_spatial_damaturu_comb_2020.tif")

# check coordinate system of damaturu_cf
print(f"coordinate system: {damaturu_cf.crs}")

coordinate system: EPSG:4326


In [23]:
# check coordinate system of aoi, damaturu.shp
aoi = gpd.read_file("data/raw/AOI/damaturu.shp")
print(f"coordinate system: {aoi.crs}")

coordinate system: EPSG:4326


In [24]:
# convert damaturu_cf from tif to GeoJSON for export and viz
damaturu_cf_geojson = raster_to_polygons(
    "data/interim/2024-12-nigeria-damaturu_02-process-output_spatial_damaturu_comb_2020.tif",
    "data/raw/AOI/damaturu.shp",
    "data/processed/damaturu_cf.geojson")

Original data type: uint8


# 10.0 Surface Temperature

In [13]:
# save Damaturu surface temperature GCP tif as variable
damaturu_st = rasterio.open("data/interim/2024-12-nigeria-damaturu_02-process-output_spatial_damaturu_summer.tif")

# check coordinate system of damaturu_st
print(f"coordinate system: {damaturu_st.crs}")

coordinate system: EPSG:4326


In [12]:
# check coordinate system of aoi, damaturu.shp
aoi = gpd.read_file("data/raw/AOI/damaturu.shp")
print(f"coordinate system: {aoi.crs}")

coordinate system: EPSG:4326


In [14]:
# convert damaturu_st from tif to GeoJSON for export and viz
damaturu_st_geojson = raster_to_polygons(
    "data/interim/2024-12-nigeria-damaturu_02-process-output_spatial_damaturu_summer.tif",
    "data/raw/AOI/damaturu.shp",
    "data/processed/damaturu_st.geojson"
)

Original data type: float64


# 11.0 Conflict Events

In [83]:
# read in ACLED damaturu shapefile
damaturu_acled = gpd.read_file("data/interim/Acled_Damaturu_LGA.shp")

# check coordinate system of damaturu_acled
print(f"coordinate system: {damaturu_acled.crs}")

coordinate system: EPSG:4326


In [84]:
# read in isochrone ACLED damaturu shapefile
damaturu_acled_isochrone = gpd.read_file("data/interim/Isochrones_Acled.shp")

# check coordinate system of damaturu_acled_isochrone
print(f"coordinate system: {damaturu_acled_isochrone.crs}")

coordinate system: EPSG:4326


In [85]:
# read in isochrone IDP site damaturu shapefile
damaturu_idp_isochrone = gpd.read_file("data/interim/Isochrones_IDP sites.shp")

# check coordinate system of damaturu_idp_isochrone
print(f"coordinate system: {damaturu_idp_isochrone.crs}")

coordinate system: EPSG:4326


In [7]:
# gain insight into damaturu_acled
with pd.option_context('display.max_rows', 5, 'display.max_columns', None): 
    display(damaturu_acled)

,event_id_c,event_date,year,time_preci,disorder_t,event_type,sub_event_,actor1,assoc_acto,inter1,actor2,assoc_ac_1,inter2,interactio,civilian_t,iso,region,country,admin1,admin2,admin3,location,latitude,longitude,geo_precis,source,source_sca,notes,fatalities,tags,timestamp,geometry
0,NIG38756,30 September 2024,2024,2,Political violence,Battles,Armed clash,Military Forces of Nigeria (2023-),None,State forces,Islamic State West Africa Province (ISWAP) and...,None,Rebel group,State forces-Rebel group,None,566,Western Africa,Nigeria,Yobe,Damaturu,None,Sasawa Kabaru,11.8371,11.9858,1,Daily Post (Nigeria),National,"Around 30 September 2024 (as reported), the Ni...",1,None,1730156965,POINT (11.98580 11.83710)
1,NIG38416,02 August 2024,2024,1,Demonstrations,Protests,Protest with intervention,Protesters (Nigeria),None,Protesters,Police Forces of Nigeria (2023-),None,State forces,State forces-Protesters,None,566,Western Africa,Nigeria,Yobe,Damaturu,None,Damaturu,11.7470,11.9608,1,"Risk and Strategic Management, Corporation",Local partner-International,"On 2 August 2024, protesters gathered in front...",0,crowd size=no report,1725924069,POINT (11.96080 11.74700)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,NIG5854,16 May 2014,2014,1,Political violence,Violence against civilians,Attack,Boko Haram - Jamaatu Ahli is-Sunnah lid-Dawati...,None,Rebel group,Civilians (Nigeria),None,Civilians,Rebel group-Civilians,Civilian targeting,566,Western Africa,Nigeria,Yobe,Damaturu,None,Damaturu,11.7470,11.9608,2,Daily Leadership (Nigeria),National,Boko Haram gunmen attack a vehicle along the M...,9,None,1702343756,POINT (11.96080 11.74700)
77,NIG5774,04 May 2014,2014,1,Strategic developments,Strategic developments,Disrupted weapons use,Unidentified Armed Group (Nigeria),None,Political militia,Police Forces of Nigeria (1999-2015),None,State forces,State forces-Political militia,None,566,Western Africa,Nigeria,Yobe,Damaturu,None,Damaturu,11.7470,11.9608,1,Daily Leadership (Nigeria),National,Defusal: Police intersect a would-be suicide b...,0,None,1702343756,POINT (11.96080 11.74700)


In [10]:
# print the column names of damaturu_acled
print(damaturu_acled.columns)

Index(['event_id_c', 'event_date', 'year', 'time_preci', 'disorder_t',
       'event_type', 'sub_event_', 'actor1', 'assoc_acto', 'inter1', 'actor2',
       'assoc_ac_1', 'inter2', 'interactio', 'civilian_t', 'iso', 'region',
       'country', 'admin1', 'admin2', 'admin3', 'location', 'latitude',
       'longitude', 'geo_precis', 'source', 'source_sca', 'notes',
       'fatalities', 'tags', 'timestamp', 'geometry'],
      dtype='object')


In [12]:
# save damaturu_acled to GEOJSON for export and viz
damaturu_acled.to_file("data/processed/damaturu_acled.geojson", driver='GeoJSON')

In [88]:
# gain insight into damaturu_acled_isochrone
with pd.option_context('display.max_rows', 15, 'display.max_columns', None): 
    display(damaturu_acled_isochrone)

,ID,CENTER_LON,CENTER_LAT,AA_METERS,AA_MODE,TOTAL_POP,geometry
0,None,11.960727420131132,11.747163119375834,5000,driving-car,111210,"POLYGON ((11.92080 11.74596, 11.92100 11.74945..."
1,None,11.960727420131132,11.747163119375834,1000,driving-car,12440,"POLYGON ((11.95150 11.74484, 11.95134 11.74549..."
2,None,11.960727420131132,11.747163119375834,5000,driving-car,111210,"POLYGON ((11.92080 11.74596, 11.92100 11.74945..."
3,None,11.960727420131132,11.747163119375834,1000,driving-car,12440,"POLYGON ((11.95150 11.74484, 11.95134 11.74549..."
4,None,11.960727420131132,11.747163119375834,5000,driving-car,111210,"POLYGON ((11.92080 11.74596, 11.92100 11.74945..."
...,...,...,...,...,...,...,...
129,None,11.960727420131132,11.747163119375834,1000,driving-car,12440,"POLYGON ((11.95150 11.74484, 11.95134 11.74549..."
130,None,11.960727420131132,11.747163119375834,5000,driving-car,111210,"POLYGON ((11.92080 11.74596, 11.92100 11.74945..."
131,None,11.960727420131132,11.747163119375834,1000,driving-car,12440,"POLYGON ((11.95150 11.74484, 11.95134 11.74549..."
132,None,11.960727420131132,11.747163119375834,5000,driving-car,111210,"POLYGON ((11.92080 11.74596, 11.92100 11.74945..."


In [ ]:
# save damaturu_acled_isochrone to GEOJSON for export and viz
damaturu_acled_isochrone.to_file("data/processed/damaturu_acled_isochrone.geojson", driver='GeoJSON')

In [90]:
# check the values in "damaturu_acled_isochrone" for "AA_METERS"
print(damaturu_acled_isochrone['AA_METERS'].unique())

[5000 1000]


In [93]:
# create new dataframe, "damaturu_acled_ischrone_1000" that only includes rows where "AA_METERS" is "1000"
damaturu_acled_ischrone_1000 = damaturu_acled_isochrone[damaturu_acled_isochrone['AA_METERS'] == 1000]

In [100]:
# gain insight into damaturu_acled_isochrone_1000
with pd.option_context('display.max_rows', 15, 'display.max_columns', None): 
    display(damaturu_acled_ischrone_1000)

,ID,CENTER_LON,CENTER_LAT,AA_METERS,AA_MODE,TOTAL_POP,geometry
1,None,11.960727420131132,11.747163119375834,1000,driving-car,12440,"POLYGON ((11.95150 11.74484, 11.95134 11.74549..."
3,None,11.960727420131132,11.747163119375834,1000,driving-car,12440,"POLYGON ((11.95150 11.74484, 11.95134 11.74549..."
5,None,11.960727420131132,11.747163119375834,1000,driving-car,12440,"POLYGON ((11.95150 11.74484, 11.95134 11.74549..."
7,None,11.960727420131132,11.747163119375834,1000,driving-car,12440,"POLYGON ((11.95150 11.74484, 11.95134 11.74549..."
9,None,11.960727420131132,11.747163119375834,1000,driving-car,12440,"POLYGON ((11.95150 11.74484, 11.95134 11.74549..."
...,...,...,...,...,...,...,...
125,None,11.960727420131132,11.747163119375834,1000,driving-car,12440,"POLYGON ((11.95150 11.74484, 11.95134 11.74549..."
127,None,11.960727420131132,11.747163119375834,1000,driving-car,12440,"POLYGON ((11.95150 11.74484, 11.95134 11.74549..."
129,None,11.960727420131132,11.747163119375834,1000,driving-car,12440,"POLYGON ((11.95150 11.74484, 11.95134 11.74549..."
131,None,11.960727420131132,11.747163119375834,1000,driving-car,12440,"POLYGON ((11.95150 11.74484, 11.95134 11.74549..."


In [101]:
# save damaturu_acled_isochrone_1000 to GEOJSON for export and viz
damaturu_acled_ischrone_1000.to_file("data/processed/damaturu_acled_isochrone_1000.geojson", driver='GeoJSON')

In [97]:
# create new dataframe, "damaturu_acled_ischrone_5000" that only includes rows where "AA_METERS" is "5000"
damaturu_acled_ischrone_5000 = damaturu_acled_isochrone[damaturu_acled_isochrone['AA_METERS'] == 5000]

In [98]:
# gain insight into damaturu_acled_isochrone_5000
with pd.option_context('display.max_rows', 15, 'display.max_columns', None): 
    display(damaturu_acled_ischrone_5000)

,ID,CENTER_LON,CENTER_LAT,AA_METERS,AA_MODE,TOTAL_POP,geometry
0,None,11.960727420131132,11.747163119375834,5000,driving-car,111210,"POLYGON ((11.92080 11.74596, 11.92100 11.74945..."
2,None,11.960727420131132,11.747163119375834,5000,driving-car,111210,"POLYGON ((11.92080 11.74596, 11.92100 11.74945..."
4,None,11.960727420131132,11.747163119375834,5000,driving-car,111210,"POLYGON ((11.92080 11.74596, 11.92100 11.74945..."
6,None,11.960727420131132,11.747163119375834,5000,driving-car,111210,"POLYGON ((11.92080 11.74596, 11.92100 11.74945..."
8,None,11.960727420131132,11.747163119375834,5000,driving-car,111210,"POLYGON ((11.92080 11.74596, 11.92100 11.74945..."
...,...,...,...,...,...,...,...
124,None,11.960727420131132,11.747163119375834,5000,driving-car,111210,"POLYGON ((11.92080 11.74596, 11.92100 11.74945..."
126,None,11.960727420131132,11.747163119375834,5000,driving-car,111210,"POLYGON ((11.92080 11.74596, 11.92100 11.74945..."
128,None,11.960727420131132,11.747163119375834,5000,driving-car,111210,"POLYGON ((11.92080 11.74596, 11.92100 11.74945..."
130,None,11.960727420131132,11.747163119375834,5000,driving-car,111210,"POLYGON ((11.92080 11.74596, 11.92100 11.74945..."


In [102]:
# save damaturu_acled_isochrone_5000 to GEOJSON for export and viz
damaturu_acled_ischrone_5000.to_file("data/processed/damaturu_acled_isochrone_5000.geojson", driver='GeoJSON')

In [89]:
# gain insight into damaturu_idp_isochrone
with pd.option_context('display.max_rows', 15, 'display.max_columns', None): 
    display(damaturu_idp_isochrone)

,ID,CENTER_LON,CENTER_LAT,AA_METERS,AA_MODE,TOTAL_POP,geometry
0,None,12.012146250061813,11.739049176025542,5000,driving-car,25410,"POLYGON ((11.96724 11.74366, 11.96718 11.74403..."
1,None,12.012146250061813,11.739049176025542,1000,driving-car,2204,"POLYGON ((12.00392 11.73907, 12.00281 11.73967..."
2,None,11.981064449803247,11.729805956312811,5000,driving-car,80922,"POLYGON ((11.94359 11.73261, 11.94458 11.73653..."
3,None,11.981064449803247,11.729805956312811,1000,driving-car,1130,"POLYGON ((11.97468 11.73164, 11.97474 11.73524..."
4,None,11.980538248965175,11.729498856331274,5000,driving-car,82099,"POLYGON ((11.94235 11.72808, 11.94337 11.73330..."
5,None,11.980538248965175,11.729498856331274,1000,driving-car,1185,"POLYGON ((11.97448 11.73164, 11.97454 11.73524..."
6,None,12.035147040636858,11.726606434528414,5000,driving-car,5564,"POLYGON ((11.99697 11.73545, 11.99697 11.73547..."
7,None,12.035147040636858,11.726606434528414,1000,driving-car,237,"POLYGON ((12.03029 11.73130, 12.03027 11.73166..."


In [87]:
# save damaturu_idp_isochrone to GEOJSON for export and viz
damaturu_idp_isochrone.to_file("data/processed/damaturu_idp_isochrone.geojson", driver='GeoJSON')